In [1]:
import warnings
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from torch.nn.utils import clip_grad_norm_
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from transformers import BertTokenizer, DistilBertTokenizer, BertForSequenceClassification, DistilBertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from tqdm import trange
warnings.filterwarnings('ignore')

In [2]:
for root, dirs, files in os.walk('C://emotions_dataset/'):
    for file in files:
        file_path = os.path.join(root, file)
        print(file_path)

C://emotions_dataset/test.txt
C://emotions_dataset/train.txt
C://emotions_dataset/val.txt


In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_gpu = torch.cuda.device_count()

In [4]:
train_data = pd.read_csv('C://emotions_dataset/train.txt', delimiter=';', names=['Sentence', 'Reaction'])
val_data = pd.read_csv('C://emotions_dataset/val.txt', delimiter=';', names=['Sentence', 'Reaction'])
test_data = pd.read_csv('C://emotions_dataset/test.txt', delimiter=';', names=['Sentence', 'Reaction'])
data = pd.concat([train_data, val_data, test_data], axis=0).reset_index(drop=True)
emotions = data['Reaction'].unique()
emotions.sort()
print('Types of emotion: ', emotions)
label_encoder = LabelEncoder()
label_encoder.fit(data['Reaction'].unique())
labels = pd.DataFrame(label_encoder.transform(data['Reaction']), columns=['Label'])
data = pd.concat([data.drop('Reaction', axis=1), labels], axis=1)

Types of emotion:  ['anger' 'fear' 'joy' 'love' 'sadness' 'surprise']


In [5]:
print(data.Label.value_counts().sort_index())
texts = data.Sentence.values
labels = data.Label.values
fact_max_len = max([len(i) for i in texts])
print('The biggest length among all sentences: ', fact_max_len)
MAX_LEN = 512
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)
tokenizer_transfrom = [tokenizer.encode(text, max_length=MAX_LEN, truncation=True, add_special_tokens=True, padding='max_length') for text in texts]
attention_mask = [[float(token>0) for token in sentence] for sentence in tokenizer_transfrom]
print('Sentence before tokenization: ', texts[777])
print('Sentence after tokenization: ', tokenizer_transfrom[777])
print('Attention mask for sentence after tokenization: ', attention_mask[777])

Label
0    2709
1    2373
2    6761
3    1641
4    5797
5     719
Name: count, dtype: int64
The biggest length among all sentences:  300
Sentence before tokenization:  i sometimes feel resentful that this has come into our lives at this time
Sentence after tokenization:  [101, 1045, 2823, 2514, 24501, 4765, 3993, 2008, 2023, 2038, 2272, 2046, 2256, 3268, 2012, 2023, 2051, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(tokenizer_transfrom, labels, test_size=0.15, random_state=777)
MASK_train, MASK_test, _, _ = train_test_split(attention_mask, tokenizer_transfrom, test_size=0.15, random_state=777)

X_train = torch.tensor(X_train)
X_test = torch.tensor(X_test)
Y_train = torch.tensor(Y_train).type(torch.LongTensor)
Y_test = torch.tensor(Y_test).type(torch.LongTensor)
MASK_train = torch.tensor(MASK_train)
MASK_test = torch.tensor(MASK_test)

batch_size = 32

train_data = TensorDataset(X_train, MASK_train, Y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

test_data = TensorDataset(X_test, MASK_test, Y_test)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

print('Element of train data: ', train_data[999])
print('Element of test data: ', test_data[999])


Element of train data:  (tensor([  101,  1045,  2562,  6603,  2339, 10047,  7294,  3681,  1997,  9940,
         1998,  3279,  2130,  2096, 10047,  2383,  4569,  2030,  3110,  7568,
         2030,  9107,  2070,  6919,  2814,  1998,  3653,  2621,  2051,  6322,
          102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,    

In [7]:
#model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6).to(device)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=6).to(device)

learning_rate = 1e-5
adamw_eps = 1e-8
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adamw_eps)

epochs = 3
train_period = len(train_dataloader)*epochs
warmup_period = int(train_period*0.1)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_period, num_training_steps=train_period)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'classifier.w

In [ ]:
# Train and validate model
    # Training part
train_loss_history = []
model.zero_grad()
for _ in trange(1, epochs+1, desc='Epoch'):
    print("\n" + "$"*20 + f"     Epoch № {_}     " + "$"*20)
    epoch_loss = 0
    for iter, batch in enumerate(train_dataloader):
        model.train()
        batch = [i.to(device) for i in batch]
        sentence_from_batch, mask_from_batch, label_from_batch = batch
        output = model(sentence_from_batch, attention_mask=mask_from_batch, labels=label_from_batch)
        loss = output[0]
        loss.backward()
        clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        epoch_loss += loss.item()
    avg_epoch_loss = epoch_loss / len(train_dataloader)
    train_loss_history.append(avg_epoch_loss)
    print("\nCurrent average training loss: ", train_loss_history[-1])

    # Validation part

    model.eval()
    accuracy, eval_iters = 0, 0
    for batch in test_dataloader:
        batch = [i.to(device) for i in batch]
        eval_batch_sentence, eval_batch_mask, eval_batch_labels = batch
        with torch.no_grad():
            output = model(eval_batch_sentence, attention_mask=eval_batch_mask)
        logits = output[0].to('cpu').numpy()
        eval_batch_labels = eval_batch_labels.to('cpu').numpy()
        Y_pred = np.argmax(logits, axis=1).flatten()
        Y_true = eval_batch_labels.flatten()
        accuracy += balanced_accuracy_score(Y_true, Y_pred)
        eval_iters += 1
        metric = pd.DataFrame({'Epoch': epochs, 'True class': Y_true, 'Pred class': Y_pred})
    print("Test accuracy: ", accuracy / eval_iters)

Epoch:   0%|                                                                                     | 0/3 [00:00<?, ?it/s]


$$$$$$$$$$$$$$$$$$$$     Epoch № 1     $$$$$$$$$$$$$$$$$$$$


Epoch:   0%|                                                                                     | 0/3 [01:26<?, ?it/s]


In [ ]:
model_save_folder = 'model/distial_bert_diplom5'
tokenizer_save_folder = 'tokenizer/distial_bert_diplom5'
model.save_pretrained(model_save_folder)
tokenizer.save_pretrained(tokenizer_save_folder)